In [1]:
import numpy as np
from numpy.linalg import pinv
import matplotlib.pyplot as plt

from sklearn import datasets 
from sklearn.model_selection import train_test_split

import pandas as pd
import seaborn as sns

In [2]:
# grad the dataset from sklearn datasets
in_data = datasets.load_breast_cancer()
# store features and data points (569 data points X 30 features)
df = pd.DataFrame(in_data.data, columns=in_data.feature_names)
# store data in array
data = df.values
# store binary diagnoses for 569 data points (0=malignant, 1=benign)
targets = in_data['target']

In [ ]:
###################
# helper functions
###################

def init_weights(data, targets):
    X = data
    y = targets
    # add bias term x0=1 to the data
    X = np.column_stack((np.ones(X.shape[0]), X))
    # initialize weights using linear regression
    X_dag = pinv(X)
    w = np.matmul(X_dag, y)

    expected_shape = (X.shape[1],)

    # Check the shape of the result
    if w.shape != expected_shape:
        raise ValueError(f"Output has shape {result.shape}, but expected shape is {expected_shape}.")
    
    return X, w

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def poly_transform(X, degree: int=2):
    # Create the PolynomialFeatures transformer
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Transform the data
    X_poly = poly.fit_transform(X)
    
    # Print the shape of the transformed feature space
    print(X_poly.shape)  # Output will be (100, 495)

    return X_poly

def regularization():

def separate_test(data, targets, num_test_elems: int=69, random_state: int=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    X = data
    num_remove = num_test_elems
    # store indices for removal from data and use as test set
    remove_indices = np.random.choice(np.arange(X.shape[0]), size=num_remove, replace=False)
    X_test = X[remove_indices, :]    # shape (69,20)
    y_test = targets[remove_indices]   # shape (69,)
    
    # remove the test set from the training data
    X = np.delete(X, remove_indices, axis=0)   # shape (500, 20)
    y = np.delete(targets, remove_indices, axis=0)   # shape (500,)
    
    return X, X_test, y, y_test

def create_train_val(X, y, N_arr, random_state: int=None):
    if random_state is not None:
        np.random.seed(random_state)
    # split the training data into 10 sample sets with training and validation sets
    X_train_set = []
    y_train_set = []
    for num in N_arr:
        if num < 500:
            X_train, _, y_train, _ = train_test_split(X, y, train_size=num, random_state=42)
        else: 
            X_train = X; y_train = y
            
        X_train_set.append(X_train); y_train_set.append(y_train);

    return X_train_set, y_train_set

#########################
# model helper functions
#########################
def predict(X, w):
    return np.sign(np.dot(X,w))

def calculate_Ein(X, y, w):
    predictions = predict(X, w)
    return np.sum(predictions != y) / len(y)

def plot_results(Ein, Eout, N, title, filename):

    plt.plot(N, Ein*100, label='Ein', marker='o')
    plt.plot(N, Eout*100, label='Eout', marker='o')
    plt.xlabel('Size of Training Data Set')
    plt.ylabel('Error (%)')
    plt.title(title)
    plt.xticks(N, label=N)
    plt.tick_params(axis='both', direction='in')
    plt.grid(alpha=0.4)
    plt.legend()
    plt.savefig(filename)
    plt.plot()

###################
# pocket algorithm
###################
def pocket(X, y, w, max_iters: int = 1000, random_state: int=None):
    if random_state is not None:
        np.random.seed(random_state)
    '''
    Vectorized version of the pocket algorithm 
    based on the algorithm outlined in Learning from Data S3.1, pg 80.
    
    Args:
        X: set of input data points (N x d array)
        y: true value of output (N, array)
        w: initial weights (d, array)
        max_iters: maximum number of iterations before exiting

    Returns:
        pocket_w: pocket weights associated with the model
        min_Ein: the minimum error of the pocket weights
    '''
    N, d = X.shape  # N: number of points, d: number of features
    pocket_w = w.copy()  # Set pocket weight vector to initial weights
    Ein_min = calculate_Ein(X, y, w)  # Initialize pocket error as current error

    for t in range(max_iters):
        # Vectorized prediction
        predictions = np.sign(X @ w)
        # Find misclassified points
        misclassified = np.where(predictions != y)[0]

        # Stop if no misclassified points (PLA converged)
        if misclassified.size == 0:
            print(f'PLA converged after {t} iterations.')
            break

        # Randomly pick a misclassified point and update weights
        random_idx = np.random.choice(misclassified)
        w += y[random_idx] * X[random_idx]  # PLA update

        # Evaluate Ein for the new weight vector w(t+1)
        current_Ein = calculate_Ein(X, y, w)

        # If w(t+1) is better than pocket_w, update pocket_w
        if current_Ein < Ein_min:
            pocket_w = w.copy()  # Update pocket weights
            Ein_min = current_Ein  # Update minimum Ein

    # Return the best pocket weights
    return pocket_w, Ein_min

################
# evaluation 
################

def calculate_Ein_vals(X, y, w, N: int=10):
    best_weights = []
    E_ins = []
    for i in range(N):
        best_weight, Ein = pocket(X[i], y[i], w)
        best_weights.append(best_weight)
        E_ins.append(Ein)

    return np.array(E_ins), np.array(best_weights)

def calculate_error(X_set, y_set, best_weights):
    """
    Calculate errors and number of misclassified points for each subset.
    
    Args:
        X_val: List or array-like of shape (10, N, n) containing validation data subsets.
        y_val: List or array-like of shape (10, N) containing true labels for each subset.
        best_weights: List of weight arrays, one for each subset.
        
    Returns:
        num_misclassified: NumPy array of number of misclassified points for each subset.
        val_errors: NumPy array of validation error rates for each subset.
    """
    num_misclassified = []
    errors = []
    
    # Iterate over each subset of validation data
    for i in range(len(X_set)):
        predictions = predict(X_set[i], best_weights[i])
        missed = np.sum(predictions != y_set[i])  # Count how many predictions are incorrect
        error = missed / len(y_set[i])  # Calculate the error rate
        
        # Store the results for each subset
        num_misclassified.append(missed)
        errors.append(error)
    
    return np.array(num_misclassified), np.array(errors)

def calculate_Eouts(X_test, y_test, best_weights):
    num_misclassified = []
    test_errors = []
    for i in range(len(best_weights)):
        predictions = predict(X_test, best_weights[i])
        missed = np.sum(predictions != y_test)
        test_error = missed / len(y_test)

        num_misclassified.append(missed)
        test_errors.append(test_error)

    return np.array(missed), np.array(test_errors)


############################
# function to run the model
# and output results
############################

def run_model(data, targets, title, filename, random_state: int=42):
    np.random.seed(random_state)
    
    ## array of data set sizes
    N = np.array([10,50,100,200,250,300,350,400,450,500])
    
    ####################
    #  prep data
    ####################
    # initialize the weights using linear regression
    X_bias, w = init_weights(data, targets)
    # separate out the test data
    X, X_test, y, y_test = separate_test(X_bias, targets)
    # create training and validation sets
    X_train_set, y_train_set = create_train_val(X, y, N)

    #########################
    # calculate Ein and Eout
    #########################
    Eins, best_weights = calculate_Ein_vals(X_train_set, y_train_set, w)
    num_misses, Eouts = calculate_Eouts(X_test, y_test, best_weights)
    print('N: Ein   Eout')
    for i, n in enumerate(N):
        print(f'{n}: {100*Eins[i]:.02f}%  {100*Eouts[i]:.02f}%')

    #########################
    # plot results
    #########################
    plot_results(Eins, Eouts, N, title, filename)